# 使用搜狗新闻语料训练三种词向量 Word2Vec,GloVe及FastText

In [9]:
import gensim
from gensim.models import Word2Vec
class GloVe(object):
    def __init__(self,filename):
        self.filename=filename
        self.vocab_size=100
        self.dst_filename=filename+'.glove.vec'
        
    def train(self):
        import gensim,os
        if os.path.exists(self.filename)==False:
            print('文件不存在',self.filename)
        else:
            !bash tools/GloVe.sh {self.filename} {self.vocab_size}
            
        self._format2gensim(self.dst_filename)
        model=gensim.models.Word2Vec.load_word2vec_format(self.dst_filename)
        print('运行完毕，向量文件保存到:',self.dst_filename)
        return model
    
    def _format2gensim(self,dst_fname):
        src_fname='tools/GloVe/vectors.txt'
        with open(src_fname,encoding='utf8') as src:
            vector_size=len(src.readline().strip().split())-1
            vocab_size=1+len(src.readlines())
        with open(dst_fname,'w',encoding='utf8') as dst:
            dst.write('%d %d\n'%(vocab_size,vector_size))
            with open(src_fname,encoding='utf8') as src:
                for line in src:
                    dst.write(line)
        print(dst_fname,'已转换为gensim格式')
        
class FastText(object):
    def __init__(self,filename):
        self.filename=filename
        self.vocab_size=100
        self.dst_filename=filename+'.fasttext.vec'
        
    def train(self):
        import gensim,os
        if os.path.exists(self.filename)==False:
            print('文件不存在',self.filename)
        else:
            !bash tools/fastText.sh {self.filename} {self.vocab_size} {self.dst_filename[:-4]}
        
        model=gensim.models.Word2Vec.load_word2vec_format(self.dst_filename)
        print('运行完毕，向量文件保存到:',self.dst_filename)
        return model

# Word2Vec

In [10]:
from datasets import SogouCACorpus
filename='data/news_tensite_xml.smarty.dat'
sentences=SogouCACorpus(filename)
w2v_model=gensim.models.Word2Vec(sentences)

# GloVe

In [11]:
filename='data/news_tensite_xml.smarty.dat.jiebaresult'
glove=GloVe(filename)
glove_model=glove.train()

mkdir -p build
gcc src/glove.c -o build/glove -lm -pthread -Ofast -march=native -funroll-loops -Wno-unused-result
gcc src/shuffle.c -o build/shuffle -lm -pthread -Ofast -march=native -funroll-loops -Wno-unused-result
gcc src/cooccur.c -o build/cooccur -lm -pthread -Ofast -march=native -funroll-loops -Wno-unused-result
gcc src/vocab_count.c -o build/vocab_count -lm -pthread -Ofast -march=native -funroll-loops -Wno-unused-result
BUILDING VOCABULARY
Processed 0 tokens.Processed 53210 tokens.
Counted 10615 unique words.
Truncating vocabulary at min count 5.
Using vocabulary of size 1652.

COUNTING COOCCURRENCES
window size: 15
context: symmetric
max product: 13752509
overflow length: 38028356
Reading vocab from file "vocab.txt"...loaded 1652 words.
Building lookup table...table contains 2729105 elements.
Processing token: 0Processed 53210 tokens.
Writing cooccurrences to disk......2 files in total.
Merging cooccurrence files: processed 0 lines.0 lines.100000 lines.200000 lines.300000 lines

# FastText

In [12]:
filename='data/news_tensite_xml.smarty.dat.jiebaresult'
fastText=FastText(filename)
fast_model=fastText.train()

make: 对“opt”无需做任何事。
Read 0M words
Number of words:  1653
Number of labels: 0
Progress: 100.0%  words/sec/thread: 57953  lr: 0.000000  loss: 2.669541  eta: 0h0m 
运行完毕，向量文件保存到: data/news_tensite_xml.smarty.dat.jiebaresult.fasttext.vec
